In [1]:
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.image import resize

In [18]:
data = pd.read_csv("UCF-crime-frames-descriptions-partitions.csv")
train = data.loc[data["partition"] == "train"]
val = data.loc[data["partition"] == "validation"]
test = data.loc[data["partition"] == "test"]

In [33]:
labels = {"Normal": 0,
          "Abuse": 1,
          "Arrest": 2,
          "Arson": 3,
          "Burglary": 4,
          "Explosion": 5,
          "Fighting": 6,
          "RoadAccidents": 7,
          "Shooting": 8,
          "Vandalism": 9
         }

In [9]:
preprocess_input = tf.keras.applications.densenet.preprocess_input

In [57]:
class img_seq(Sequence):

    def __init__(self, x_set, y_set, directorys, batch_size, path):
        self.x = list(x_set)
        self.y = list(y_set)
        self.y = np.array([labels[lbl] for lbl in self.y])
        self.batch_size = batch_size
        self.dirs = list(directorys)
        self.path = path
        
    def __len__(self):
        return len(self.x) // self.batch_size

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.x))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]       
        batch_dirs = self.dirs[low:high]
        images = []
        for d, file in zip(batch_dirs, batch_x):
            im = tf.keras.preprocessing.image.load_img(self.path + d + "/" + file)
            im = tf.keras.preprocessing.image.img_to_array(im)            
            im = resize(im, (64, 64), method="bilinear")            
            images.append(im)            
        images = self.pre_process_img(np.array(images))
        return images, np.array(batch_y)
    
    def pre_process_img(self, X):
        return preprocess_input(X)

In [58]:
path = "d:/data/UCF-crime/Anomaly-Videos-Frames/"
X = img_seq(train["frame"], train["class"], train["directory"], 64, path)
V = img_seq(val["frame"], val["class"], val["directory"], 64, path)

In [63]:
dense = tf.keras.applications.DenseNet121(input_shape=(64, 64, 3),
                                          include_top=False,
                                          weights="imagenet")
    
    
inputs = tf.keras.layers.Input(shape=(64, 64,3))
x = dense(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.3)(x)
#x = tf.keras.layers.Dense(1024, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dropout(0.4) (x)
x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)

model = tf.keras.Model(inputs=inputs, outputs = x)
dense.trainable = False

In [65]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy',
              metrics = ["acc"])    

In [76]:
history = model.fit(X, validation_data=V, epochs=10)

Epoch 1/10
1939/1939 [==============================] - 634s 327ms/step - loss: 0.8884 - acc: 0.7376 - val_loss: 1.3050 - val_acc: 0.6620
Epoch 2/10
1939/1939 [==============================] - 634s 327ms/step - loss: 0.7131 - acc: 0.7822 - val_loss: 1.3503 - val_acc: 0.6654
Epoch 3/10
1939/1939 [==============================] - 632s 326ms/step - loss: 0.6025 - acc: 0.8102 - val_loss: 1.4315 - val_acc: 0.6581
Epoch 4/10
1939/1939 [==============================] - 635s 327ms/step - loss: 0.5278 - acc: 0.8265 - val_loss: 1.4262 - val_acc: 0.6596
Epoch 5/10
1939/1939 [==============================] - 630s 325ms/step - loss: 0.4783 - acc: 0.8400 - val_loss: 1.6345 - val_acc: 0.6635
Epoch 6/10
1939/1939 [==============================] - 631s 326ms/step - loss: 0.4386 - acc: 0.8510 - val_loss: 1.5396 - val_acc: 0.6616
Epoch 7/10
1939/1939 [==============================] - 636s 328ms/step - loss: 0.4013 - acc: 0.8614 - val_loss: 1.6177 - val_acc: 0.6586
Epoch 8/10
1939/1939 [============

In [77]:
model.save("dense.h5")

In [78]:
val_pred = model.predict(V)

In [79]:
print(classification_report(V.y[0:19328], tf.argmax(val_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.67      0.97      0.79     12893
           1       0.14      0.03      0.05       646
           2       0.01      0.00      0.00      1321
           3       0.02      0.02      0.02       106
           4       0.43      0.03      0.06      1767
           5       0.47      0.05      0.09       388
           6       0.01      0.00      0.00      1132
           7       0.00      0.00      0.00        12
           8       0.00      0.00      0.00        44
           9       0.00      0.00      0.00      1019

    accuracy                           0.65     19328
   macro avg       0.17      0.11      0.10     19328
weighted avg       0.50      0.65      0.54     19328



In [80]:
y_pred = model.predict(X)

In [81]:
print(classification_report(X.y[0:len(y_pred)], tf.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93     84646
           1       0.71      0.77      0.74      2593
           2       0.90      0.97      0.93     10454
           3       0.93      0.93      0.93      4984
           4       0.91      0.89      0.90      8211
           5       0.93      0.74      0.83      2138
           6       0.85      0.83      0.84      6178
           7       0.29      0.46      0.36       689
           8       0.82      0.37      0.51      1557
           9       0.79      0.74      0.76      2646

    accuracy                           0.91    124096
   macro avg       0.81      0.76      0.77    124096
weighted avg       0.91      0.91      0.91    124096

